<a href="https://colab.research.google.com/github/SwatiNeha/neural-networks-applied/blob/main/Spherical_projection_gaussian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SPHERICAL PROJECTION OF GAUSSIAN DATA**

***1.*** ***Generating Training data using PyTorch***

In [ ]:
import torch as tp
import plotly.graph_objects as go
import numpy as np

# Setting the seed for reproducibility
seed = 42
np.random.seed(seed)
tp.manual_seed(seed)

def generate_norm_samples(data_size):
    X = tp.normal(mean=0.0, std=1.0, size=(data_size, 3))
    return X

def generate_output_sphere(data):
    y = data / tp.norm(data, dim=1, keepdim=True)
    return y

data_size = 10000
X = generate_norm_samples(data_size)
y = generate_output_sphere(X)


***2.*** ***Splitting Training data into Train Validation and Test set***

In [ ]:
from sklearn.model_selection import train_test_split

X_train_init, X_test, y_train_init, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_init, y_train_init, test_size=0.25, random_state=42)
print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(X_test)}")

Training set size: 6000
Validation set size: 2000
Test set size: 2000


***3.*** ***Plotting output of training data on Unit sphere***

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=y_train[:, 0],
    y=y_train[:, 1],
    z=y_train[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color='brown',
        opacity=0.8
    ),
    name='Training Data'
))


fig.add_trace(go.Scatter3d(
    x=[0],
    y=[0],
    z=[0],
    mode='markers',
    marker=dict(
        size=3,
        color='black',
        opacity=0.8
    ),
    name='Origin'
))

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'
    ),
    title='Training Data Output on Unit Sphere'
)

fig.show()

***4. Initiaization of Neural Network Model***

In [ ]:
import torch.nn as nn
import torch.optim as optim


class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(3, 20)
        self.layer2 = nn.Linear(20, 20)
        self.layer3 = nn.Linear(20, 3)

    def forward(self, x):
        x = tp.relu(self.layer1(x))
        x = tp.relu(self.layer2(x))
        x = self.layer3(x)
        return x

NN_model = NeuralNetwork()

# Print weights and biases
#for layer in [NN_model.layer1, NN_model.layer2, NN_model.layer3]:
    #print("Weights (W):")
    #print(layer.weight)  # Initialized randomly by default
    #print("\nBiases (b):")
    #print(layer.bias)

***5.*** ***Training vs Validation Loss on optimal hyperparameters***

In [ ]:

crit = nn.MSELoss()
optimizer = optim.Adam(NN_model.parameters(), lr=0.001)


epochs = 200
batch_size = 64

# Early stopping parameters
param_wait = 10 #for how many epochs to check no improvement in loss
best_val_loss = float('inf')
no_improve = 0 #if no improvement in loss
early_stop = False

# Training Loss
train_losses = []
valid_losses = []

for epoch in range(epochs):
    if early_stop:
        print("Stopped Early")
        break
    NN_model.train()
    choices = tp.randperm(X_train.size()[0])
    epoch_loss = 0.0

    for i in range(0, X_train.size()[0], batch_size):
        index = choices[i:i+batch_size]
        x1, y1 = X_train[index], y_train[index]

        optimizer.zero_grad()
        outputs = NN_model(x1)
        loss = crit(outputs, y1)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    train_loss = epoch_loss / (X_train.size()[0] / batch_size)
    train_losses.append(train_loss)

    # Validation loss
    NN_model.eval()
    with tp.no_grad():
        valid_outputs = NN_model(X_val)
        valid_loss = crit(valid_outputs, y_val).item()
        valid_losses.append(valid_loss)

    print(f'Epoch {epoch+1}, Train Loss: {train_loss}, Val Loss: {valid_loss}')

    if valid_loss < best_val_loss:
        best_val_loss = valid_loss
        no_improve = 0
    else:
        no_improve += 1

    if no_improve >= param_wait:
        print(f'Early stopping at epoch {epoch+1}')
        early_stop = True


Epoch 1, Train Loss: 0.24529231786727906, Val Loss: 0.10646291822195053
Epoch 2, Train Loss: 0.05495940017700195, Val Loss: 0.036332715302705765
Epoch 3, Train Loss: 0.031673944930235544, Val Loss: 0.02560129016637802
Epoch 4, Train Loss: 0.022200202266375223, Val Loss: 0.017969703301787376
Epoch 5, Train Loss: 0.015344449043273926, Val Loss: 0.012579684145748615
Epoch 6, Train Loss: 0.01074875380595525, Val Loss: 0.00897917803376913
Epoch 7, Train Loss: 0.007813757456839085, Val Loss: 0.006615578196942806
Epoch 8, Train Loss: 0.005953433878719806, Val Loss: 0.005096910987049341
Epoch 9, Train Loss: 0.004797915716965993, Val Loss: 0.004056922625750303
Epoch 10, Train Loss: 0.0040401738062500955, Val Loss: 0.003452995326370001
Epoch 11, Train Loss: 0.0035311567733685174, Val Loss: 0.0030525836627930403
Epoch 12, Train Loss: 0.003174395361294349, Val Loss: 0.002697648946195841
Epoch 13, Train Loss: 0.002910728142907222, Val Loss: 0.002483396092429757
Epoch 14, Train Loss: 0.0027151278108

***6.*** ***Training vs Validation Loss Plot***

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(range(1, epochs + 1)),
    y=train_losses,
    mode='lines',
    name='Training Loss',
    line=dict(width=8)
))

fig.add_trace(go.Scatter(
    x=list(range(1, epochs + 1)),
    y=valid_losses,
    mode='lines',
    name='Validation Loss',
    line=dict(width=8)
))

fig.update_layout(
    title=dict(
        text='Training and Validation Loss Over Epochs',
        font=dict(size=20)
    ),
    xaxis=dict(
        title=dict(
            text='Epochs',
            font=dict(size=24)
        ),
        tickfont=dict(size=24)
    ),
    yaxis=dict(
        title=dict(
            text='Loss',
            font=dict(size=24)
        ),
        tickfont=dict(size=24)
    ),
    legend=dict(
        x=100,
        y=1,
        traceorder='normal',
        font=dict(size=24)
    ),
    margin=dict(l=0, r=0, t=40, b=0)
)

# Show plot
fig.show()

***7.*** ***Evaluation on test data***

In [ ]:
NN_model.eval()
with tp.no_grad():
    y_predicted = NN_model(X_test)
    testing_loss = crit(y_predicted, y_test).item()
print(f'Test Loss: {testing_loss}')

Test Loss: 0.0008716268348507583


***8. Plotting output of testing data on Unit sphere***


In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=y_predicted[:, 0],
    y=y_predicted[:, 1],
    z=y_predicted[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color='green',
        opacity=0.8
    ),
    name='Predicted Output'
))


fig.add_trace(go.Scatter3d(
    x=[0],
    y=[0],
    z=[0],
    mode='markers',
    marker=dict(
        size=3,
        color='black',
        opacity=0.8
    ),
    name='Origin'
))

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'
    ),
    title='Predicted Testing Data Output on Unit Sphere'
)

fig.show()